In [1]:
pip install requests youtube-transcript-api pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import SRTFormatter

# Fonction pour rechercher des vidéos sur YouTube avec des mots-clés
def get_videos(query, api_key):
    url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&key={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200:
        # Extraire les IDs des vidéos de la réponse JSON
        video_data = response.json()
        video_ids = [item['id']['videoId'] for item in video_data['items']]
        return video_ids
    else:
        print(f"Erreur lors de la recherche des vidéos: {response.status_code}")
        return []

# Fonction pour obtenir les informations de la vidéo à partir de l'API YouTube
def get_video_info(video_id, api_key):
    url = f"https://www.googleapis.com/youtube/v3/videos"
    params = {
        'part': 'snippet',  # Récupère les informations de base de la vidéo (titre, description, date)
        'id': video_id,  # ID de la vidéo
        'key': api_key   # Votre clé API
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        video_data = response.json()
        if 'items' in video_data and len(video_data['items']) > 0:
            snippet = video_data['items'][0]['snippet']
            title = snippet['title']
            description = snippet['description']
            published_at = snippet['publishedAt']
            return title, description, published_at
    else:
        print(f"Erreur lors de la récupération des infos vidéo: {response.status_code}")
    return None, None, None

# Fonction pour obtenir les sous-titres d'une vidéo
def get_video_subtitles(video_id, languages=['fr']):
    try:
        # Tente de récupérer les sous-titres dans les langues spécifiées
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
        
        # Utiliser un formateur pour convertir les sous-titres en format SRT
        formatter = SRTFormatter()
        formatted_transcript = formatter.format_transcript(transcript)
        
        return formatted_transcript
    except Exception as e:
        print(f"Erreur lors de l'extraction des sous-titres pour {video_id}: {e}")
        return None

# Fonction principale pour récupérer les données et les stocker dans un DataFrame
def collect_video_data(query, api_key):
    # Étape 1: Recherche des vidéos en utilisant la fonction get_videos
    video_ids = get_videos(query, api_key)
    
    video_data = []
    
    for video_id in video_ids:
        print(f"Récupération des données pour la vidéo {video_id}...")
        
        # Étape 2: Récupérer les informations de chaque vidéo
        title, description, published_at = get_video_info(video_id, api_key)
        
        if title and description and published_at:
            # Étape 3: Récupérer les sous-titres si disponibles
            subtitles = get_video_subtitles(video_id)
            
            # Étape 4: Stocker les données dans une liste
            video_data.append({
                'Video ID': video_id,
                'Title': title,
                'Description': description,
                'Published At': published_at,
                'Subtitles': subtitles
            })
    
    # Convertir les données collectées en DataFrame
    df = pd.DataFrame(video_data)
    return df

# Exemple d'utilisation
api_key = "XXXXXXXXXXXXXXXXXX"  # Remplacez par sa clé API
query = "nouveauté livre 2023"

# Collecter les données et créer un DataFrame
df = collect_video_data(query, api_key)

# Sauvegarder dans un fichier CSV (facultatif)
# df.to_csv("video_data.csv", index=False)

# Filtrer les vidéos qui ont des sous-titres
df_with_subtitles = df[df['Subtitles'].notna() & (df['Subtitles'] != '')]

# Afficher les résultats par pages de 100 lignes
page_size = 100
for start in range(0, len(df_with_subtitles), page_size):
    print(df_with_subtitles.iloc[start:start + page_size])
    input("Appuyez sur Entrée pour afficher la page suivante...")


Récupération des données pour la vidéo 0-HWVOEXsv0...
Récupération des données pour la vidéo FN4Lp2CQ1Kw...
Récupération des données pour la vidéo -RTlJqOZaOM...
Récupération des données pour la vidéo msSHHUkTs_w...
Récupération des données pour la vidéo hpWeX6VET-w...
      Video ID                                              Title  \
0  0-HWVOEXsv0  TOP 10 de 2023 📚 mes meilleures lectures de l'...   
1  FN4Lp2CQ1Kw  J'ai lu 124 LIVRES cette année, voici mes favoris   
2  -RTlJqOZaOM  JE JUGE LES 50 LIVRES LES PLUS LUS DE 2023 (se...   
3  msSHHUkTs_w       ARRIVÉE LIVRESQUE 📚 +25 livres en avril 2023   
4  hpWeX6VET-w  Mes 10 meilleures lectures de 2023 (et pourquo...   

                                         Description          Published At  \
0  Quels sont les romans que j'ai le plus aimés e...  2023-12-30T09:00:00Z   
1  Hello ma team Jeannot. Je suis trop contente d...  2023-12-16T09:00:08Z   
2  Je vous donne mon avis sur les 50 livres les p...  2023-11-17T15:30:00Z   
3  

SyntaxError: 'return' outside function (3126316449.py, line 107)

In [ ]:
import pandas as pd
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import SRTFormatter

# Fonction pour rechercher des vidéos sur YouTube avec des mots-clés
def get_videos(query, api_key, max_results=50):
    video_ids = []
    next_page_token = None

    while True:
        url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&key={api_key}'
        
        if next_page_token:
            url += f'&pageToken={next_page_token}'
        
        response = requests.get(url)
        
        if response.status_code == 200:
            video_data = response.json()
            video_ids.extend([item['id']['videoId'] for item in video_data['items']])
            next_page_token = video_data.get('nextPageToken')

            if not next_page_token:
                break
        else:
            print(f"Erreur lors de la recherche des vidéos: {response.status_code}")
            break

    return video_ids

# Fonction pour obtenir les informations de la vidéo à partir de l'API YouTube
def get_video_info(video_id, api_key):
    url = f"https://www.googleapis.com/youtube/v3/videos"
    params = {
        'part': 'snippet',  # Récupère les informations de base de la vidéo (titre, description, date)
        'id': video_id,  # ID de la vidéo
        'key': api_key   # Votre clé API
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        video_data = response.json()
        if 'items' in video_data and len(video_data['items']) > 0:
            snippet = video_data['items'][0]['snippet']
            title = snippet['title']
            description = snippet['description']
            published_at = snippet['publishedAt']
            return title, description, published_at
    else:
        print(f"Erreur lors de la récupération des infos vidéo: {response.status_code}")
    return None, None, None

# Fonction pour obtenir les sous-titres d'une vidéo
def get_video_subtitles(video_id, languages=['fr']):
    try:
        # Tente de récupérer les sous-titres dans les langues spécifiées
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
        
        # Utiliser un formateur pour convertir les sous-titres en format SRT
        formatter = SRTFormatter()
        formatted_transcript = formatter.format_transcript(transcript)
        
        return formatted_transcript
    except Exception as e:
        print(f"Erreur lors de l'extraction des sous-titres pour {video_id}: {e}")
        return None

# Fonction principale pour récupérer les données et les stocker dans un DataFrame
def collect_video_data(query, api_key):
    # Étape 1: Recherche des vidéos en utilisant la fonction get_videos
    video_ids = get_videos(query, api_key)
    
    video_data = []
    
    for video_id in video_ids:
        print(f"Récupération des données pour la vidéo {video_id}...")
        
        # Étape 2: Récupérer les informations de chaque vidéo
        title, description, published_at = get_video_info(video_id, api_key)
        
        if title and description and published_at:
            # Étape 3: Récupérer les sous-titres si disponibles
            subtitles = get_video_subtitles(video_id)
            
            # Étape 4: Stocker les données dans une liste
            video_data.append({
                'Video ID': video_id,
                'Title': title,
                'Description': description,
                'Published At': published_at,
                'Subtitles': subtitles
            })
    
    # Convertir les données collectées en DataFrame
    df = pd.DataFrame(video_data)
    return df

# Exemple d'utilisation
api_key = "XXXXXXXXXXXXXXXXXXXXXXX"  # Remplacez par sa clé API
query = "nouveauté livre 2023"

# Collecter les données et créer un DataFrame
df = collect_video_data(query, api_key)

# Sauvegarder dans un fichier CSV (facultatif)
# df.to_csv("video_data.csv", index=False)

# Filtrer les vidéos qui ont des sous-titres
df_with_subtitles = df[df['Subtitles'].notna() & (df['Subtitles'] != '')]

# Sauvegarder dans un fichier CSV (facultatif)
df_with_subtitles.to_csv("video_data1.csv", index=False)

# Afficher les résultats par pages de 100 lignes
#page_size = 100
#for start in range(0, len(df_with_subtitles), page_size):
    #print(df_with_subtitles.iloc[start:start + page_size])
    #input("Appuyez sur Entrée pour afficher la page suivante...")


Récupération des données pour la vidéo 0-HWVOEXsv0...
Récupération des données pour la vidéo FN4Lp2CQ1Kw...
Récupération des données pour la vidéo -RTlJqOZaOM...
Récupération des données pour la vidéo msSHHUkTs_w...
Récupération des données pour la vidéo hpWeX6VET-w...
Récupération des données pour la vidéo bxtnqAb-4zA...
Récupération des données pour la vidéo DDLMKQfXJKc...
Erreur lors de l'extraction des sous-titres pour DDLMKQfXJKc: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=DDLMKQfXJKc! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe y